In [ ]:
import os
from crewai import Agent, Task, Crew, LLM
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain_huggingface import HuggingFaceEmbeddings
import requests

In [ ]:
#pip install -U langchain-huggingface

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install faiss-cpu

   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   ------- -------------------------------- 2.9/15.0 MB 15.2 MB/s eta 0:00:01
   ---------- ----------------------------- 3.9/15.0 MB 14.7 MB/s eta 0:00:01
   ------------- -------------------------- 5.2/15.0 MB 8.6 MB/s eta 0:00:02
   ----------------- ---------------------- 6.6/15.0 MB 8.1 MB/s eta 0:00:02
   -------------------- ------------------- 7.6/15.0 MB 7.5 MB/s eta 0:00:01
   -------------------- ------------------- 7.9/15.0 MB 7.3 MB/s eta 0:00:01
   ----------------------- ---------------- 8.7/15.0 MB 6.0 MB/s eta 0:00:02
   --------------------------- ------------ 10.2/15.0 MB 6.3 MB/s eta 0:00:01
   ------------------------------ --------- 11.3/15.0 MB 6.1 MB/s eta 0:00:01
   -------------------------------- ------- 12.3/15.0 MB 5.9 MB/s eta 0:00:01
   ---------------------------------- ----- 13.1/15.0 MB 5.7 MB/s eta 0:00:01
   ------------------------------------- -- 14.2/15.0 MB 5.8 MB/s eta 0:00:01

In [ ]:
import os

os.environ["SERPAPI_API_KEY"] = '***************************'

In [ ]:
import requests
from PyPDF2 import PdfReader
from io import BytesIO

def pdf_url_to_text(pdf_url):
    response = requests.get(pdf_url)
    if response.status_code != 200:
        raise Exception("Failed to download PDF")

    pdf_file = BytesIO(response.content)
    reader = PdfReader(pdf_file)

    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() or ""

    return full_text.strip()

In [ ]:
#paper url
pdf_url='https://bpb-us-e1.wpmucdn.com/sites.harvard.edu/dist/0/550/files/2023/11/waves_quantum.pdf'
text = pdf_url_to_text(pdf_url)[:5000]
print(text)

Chapter 10
Introduction to quantum
mechanics
David Morin, morin@physics.harvard.edu
This chapter gives a brief introduction to quantum mechanics. Quantum mechanics can be
thought of roughly as the study of physics on very small length scales, although there are
also certain macroscopic systems it directly applies to. The descriptor \quantum" arises
because in contrast with classical mechanics, certain quantities take on only discrete values.
However, some quantities still take on continuous values, as we'll see.
In quantum mechanics, particles have wavelike properties, and a particular wave equa-
tion, the Schrodinger equation , governs how these waves behave. The Schrodinger equation
is di®erent in a few ways from the other wave equations we've seen in this book. But these
di®erences won't keep us from applying all of our usual strategies for solving a wave equation
and dealing with the resulting solutions.
In some respect, quantum mechanics is just another example of a system governe

In [ ]:
def get_vectorstore_retriever(raw_text: str):
    # 1. Wrap the raw text in a Document object
    docs = [Document(page_content=raw_text)]

    # 2. Split into smaller chunks
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_docs = splitter.split_documents(docs)

    # 3. Create embeddings and FAISS index
    #embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.from_documents(split_docs, embeddings)

    # 4. Return a retriever object
    return db.as_retriever()

In [ ]:
retriever = get_vectorstore_retriever(text)

In [ ]:
def web_search(query):
    print("No relevant local data found. Falling back to web search...")
    serpapi_key = os.environ["SERPAPI_API_KEY"]
    url = f"https://serpapi.com/search.json?q={query}&api_key={serpapi_key}&num=3"

    response = requests.get(url)
    results = response.json()

    snippets = []
    if "organic_results" in results:
        for result in results["organic_results"]:
            snippet = result.get("snippet")
            if snippet:
                snippets.append(snippet)

    return "\n".join(snippets[:3]) or "No results found."


In [ ]:
# --- LLM Setup ---
llm = LLM(model="groq/llama3-70b-8192")

In [ ]:
# --- Agents ---
synthesizer = Agent(
    role="RAG Synthesizer",
    goal="Answer technical queries using documents or external sources",
    backstory="A senior AI that can write clear and accurate answers using both internal and web knowledge.",
    llm=llm,
    verbose=True
)

In [ ]:
def build_smart_rag_task(query: str, agent: Agent) -> Task:
    documents = retriever.invoke(query)
    context = "\n\n".join(doc.page_content for doc in documents)

    if len(context.strip()) < 200:
        context = web_search(query)

    return Task(
        description=f"Use the following context to answer the question:\n\n{context}\n\nQuestion: {query}",
        expected_output="A concise, accurate 3-paragraph answer.",
        agent=agent
    )


In [ ]:
# --- Crew Setup ---

query = input(" Enter your question: ").strip()
#print(f"Received query: '{query}'")
rag_task = build_smart_rag_task(query, synthesizer)
if not query:
    print(" No query provided. Exiting.")
    exit()

crew = Crew(
    agents=[synthesizer],
    tasks=[rag_task],
    verbose=True
)



 Enter your question:  Recent quantum mechanics in 2020-22


In [ ]:
result = crew.kickoff()
print("\nFinal Answer:\n", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e353df42-ed38-478d-a008-38fcbd99b02c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f53513ce-736d-442b-9cfa-087211331a1b
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f53513ce-736d-442b-9cfa-087211331a1b
       Status: Executing Task...
    └── 🤖 Agent: RAG Synthesizer
            Status: In Progress

# Agent: RAG Synthesizer
## Task: Use the following context to answer the question:

Chapter 10
Introduction to quantum
mechanics
David Morin, morin@physics.harvard.edu
This chapter gives a brief introduction to quantum mechanics. Quantum mechanics can be
thought of roughly as the study of physics on very small length scales, although there are
also certain macroscopic systems it directly applies to. The descriptor \quantum" arises
because in contrast with classical mechanics, certain quantities take on only discrete values.
However, some quantities still take on continuous values, as we'll see.
In quantum mechanics, particles have wavelike properties, and a particular wave equa-
tion, the Schrodinger equation , governs how these waves behave. The Schrodinger equation
is di®erent in a few ways from the other wave equations we've seen in this book. But these
di®erences won't keep us from applying all of our usual strategies for solving a wave equation
and dealing with the resulting solu

🚀 Crew: crew
└── 📋 Task: f53513ce-736d-442b-9cfa-087211331a1b
       Status: Executing Task...
    └── 🤖 Agent: RAG Synthesizer
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: f53513ce-736d-442b-9cfa-087211331a1b
       Assigned to: RAG Synthesizer
       Status: ✅ Completed
    └── 🤖 Agent: RAG Synthesizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f53513ce-736d-442b-9cfa-087211331a1b                                                                     │
│  Agent: RAG Synthesizer                                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e353df42-ed38-478d-a008-38fcbd99b02c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Final Answer:
 The recent developments in quantum mechanics from 2020 to 2022 are not explicitly mentioned in the provided chapter. However, based on the general understanding of quantum mechanics, some recent advancements can be discussed.

In recent years, significant progress has been made in the field of quantum computing, with tech giants like Google, IBM, and Microsoft racing to develop functional quantum computers. For instance, in October 2020, Google announced a significant breakthrough in quantum computing, claiming to have achieved "quantum supremacy" - a feat that surpasses the capabilities of classical computers. This development has opened up new avenues for research in quantum mechanics.

Furthermore, researchers have also made significant progress in understanding the principles of quantum mechanics, such as quantum entanglement and superposition. Recent studies have also explored the potential applications of quantum mechanics in fields like materials science, cryptog